In [ ]:
# 데이터셋 read
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('Target', axis=1)
train_y = train.Target

In [ ]:
train_X = pd.get_dummies(data = train_X, columns = ['Gender'], prefix = 'Gender')
test = pd.get_dummies(data = test, columns = ['Gender'], prefix = 'Gender')

In [ ]:
train_X

In [ ]:
# 전복 전체 무게 대비 불필요한 물질들의 값 ( 이물질 / 핏물 등)
foreign_body = train_X['Whole Weight'] - (train_X['Shucked Weight'] + train_X['Viscra Weight'] + train_X['Shell Weight'])

In [ ]:
foreign_body

In [ ]:
foreign_body[47]

In [ ]:
train_X['foreign body'] = foreign_body

In [ ]:
train_X.loc[train_X['foreign body'] < 0 , 'foreign body'] = 0

In [ ]:
# 테스트셋에도 적용
foreign_body = test['Whole Weight'] - (test['Shucked Weight'] + test['Viscra Weight'] + test['Shell Weight'])
test['foreign body'] = foreign_body
test.loc[test['foreign body'] < 0 , 'foreign body'] = 0

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [ ]:
# 딥러닝 모델 선언
model = Sequential()
model.add(Dense(16, input_dim=11, activation='elu'))
model.add(Dense(32, activation='elu'))    
model.add(Dense(64, activation='elu'))  
model.add(Dropout(0.5))  
model.add(Dense(32, activation='elu'))
model.add(Dense(16, activation='elu'))
model.add(Dense(1))


model.compile(loss='mean_absolute_error',
              optimizer='Nadam', 
              metrics=['mae'])

In [ ]:
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath = "./model/{epoch:02d}-{val_loss:.4f}.hdf5"

# 모델 업데이트 및 저장
cp = ModelCheckpoint(filepath=modelpath, monitor='val_mae', verbose=0, save_best_only=True, mode = 'min')

# 학습 자동 중단 설정
es = EarlyStopping(monitor='val_mae', patience=50, mode='min')

rlrp = ReduceLROnPlateau(monitor='val_mae', factor=0.2, patience=40, mode='min')

In [ ]:
# 모델 학습
model.fit(train_X, train_y, validation_split=0.3, epochs=1000, batch_size=32, verbose=1, callbacks=[es, cp, rlrp])

In [ ]:
Y_prediction = model.predict(test).flatten()

In [ ]:
Y_prediction

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['Target'] = np.round(Y_prediction)

submission

submission.to_csv('submission.csv',index=False)